In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
import folium
import branca

In [2]:
community_map = gpd.read_file('SHP\LA_SHP\Original_Shp\geo_export_d0097c2a-af0b-498c-ae6f-0f976d91920e.shp')
community_map = community_map.rename(columns={'name':'community'})

# crime data taken from https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/data
# df = pd.read_csv('./Data/LA_Crimes.csv')
# crs = {'init' : 'epsg:4326'}
# df = df.dropna(subset=['LATITUDE'])
# df= df[df['LATITUDE'] != 0]

# geometry = [Point(xy) for xy in zip(df["LONGITUDE"], df["LATITUDE"])]
# geometry[:3]


# geo_df = gpd.GeoDataFrame(df, crs = crs, geometry = geometry)

In [3]:
# taken from https://towardsdatascience.com/tagging-a-location-to-a-shapefile-area-using-geopandas-5d74336128bf
# geo_df['community'] = ''
# for idx in range(community_map.shape[0]):
#     #For every lat/long, find if they reside within a community
#     try:
#         pip = geo_df.within(community_map.loc[idx, 'geometry'])
#         if pip.sum() > 0:
#             geo_df.loc[pip, 'community']  = community_map.loc[idx, 'community']
#     except:
#         continue

In [4]:
# geo_df.to_file("LACrime.shp")

In [5]:
geo_df = gpd.read_file("SHP\LA_SHP\Summarized_Shp\LACrime.shp")

In [6]:
# taking group by and then reverting back to data frame to use
# https://technology.amis.nl/data-analytics/convert-groupby-result-on-pandas-data-frame-into-a-data-frame-using-to_frame/
crimebycommunity = geo_df.groupby(['community'])['community'].count().to_frame(name = 'count').reset_index()


crimebycommunity['community'].replace('', np.nan, inplace=True)
crimebycommunity = crimebycommunity.dropna(subset=['community'])


map_and_stats = community_map.merge(crimebycommunity, on="community")

map_and_stats_2 = map_and_stats[['community','geometry','count']]

In [27]:
# https://vverde.github.io/blob/interactivechoropleth.html

x_map = map_and_stats_2.centroid.x.mean()
y_map = map_and_stats_2.centroid.y.mean()



mymap = folium.Map(location=[y_map, x_map], zoom_start=11,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)


myscale = (map_and_stats_2['count'].quantile((0,0.5,0.9,1))).tolist()

C:\Users\Saajid\AppData\Local\Temp\ipykernel_9344\2463829639.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x_map = map_and_stats_2.centroid.x.mean()
C:\Users\Saajid\AppData\Local\Temp\ipykernel_9344\2463829639.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y_map = map_and_stats_2.centroid.y.mean()


In [28]:
mymap.choropleth(
geo_data=map_and_stats_2,
name='Choropleth',
data=map_and_stats_2,
columns=['community','count'],
key_on="feature.properties.community",
fill_color='Reds', #https://matplotlib.org/3.5.1/tutorials/colors/colormaps.html          sequential color maps
threshold_scale=myscale,
fill_opacity=0.5,
line_opacity=0.2,
legend_name='Crime Count',
smooth_factor=0
)

C:\Users\Saajid\anaconda3\envs\geopandas\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [29]:
mymap

In [30]:
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000',
                            'fillOpacity': 0.1,
                            'weight': 0.1}

highlight_function = lambda x: {'fillColor': '#000000',
                                'color':'#000000',
                                'fillOpacity': 0.50,
                                'weight': 0.1}

In [31]:
CrimeCount = folium.features.GeoJson(
    map_and_stats_2,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['community','count'],
        aliases=['Neighborhood: ','Count of Crimes '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
mymap.add_child(CrimeCount)
mymap.keep_in_front(CrimeCount)
folium.LayerControl().add_to(mymap)

In [32]:
#this map should have the interactivity now
mymap

In [33]:
df_airbnb = pd.read_csv('./Data/LA_listings.csv')


airbnb_locations = df_airbnb[['latitude','longitude']]
airbnb_list = airbnb_locations.values.tolist()

In [34]:
df_airbnb.head()

,id,listing_url,scrape_id,last_scraped,name,picture_url,host_id,host_url,host_name,host_since,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2708,https://www.airbnb.com/rooms/2708,2.020000e+13,12/6/2021,Runyon Canyon | Beau Furn Mirror Mini-Suite Fi...,https://a0.muscache.com/pictures/miso/Hosting-...,3008,https://www.airbnb.com/users/show/3008,Chas.,9/16/2008,...,4.97,5.00,4.85,NaN,t,3,0,3,0,0.69
1,3021,https://www.airbnb.com/rooms/3021,2.020000e+13,12/7/2021,Hollywood Hills Zen Modern style Apt/Guesthouse,https://a0.muscache.com/pictures/5147dcd2-efad...,3415,https://www.airbnb.com/users/show/3415,Nataraj,10/2/2008,...,4.30,4.64,4.00,NaN,f,4,1,3,0,0.23
2,5728,https://www.airbnb.com/rooms/5728,2.020000e+13,12/7/2021,Tiny Home in Artistic Oasis near Venice and LAX,https://a0.muscache.com/pictures/7a29d275-f293...,9171,https://www.airbnb.com/users/show/9171,Sanni,3/5/2009,...,4.92,4.79,4.70,HSR19-002149,f,4,1,3,0,4.25
3,5729,https://www.airbnb.com/rooms/5729,2.020000e+13,12/7/2021,Zen Room with Floating Bed near Venice and LAX,https://a0.muscache.com/pictures/f48e3ea8-2075...,9171,https://www.airbnb.com/users/show/9171,Sanni,3/5/2009,...,4.89,4.76,4.71,HSR19-002149,f,4,1,3,0,4.48
4,5843,https://www.airbnb.com/rooms/5843,2.020000e+13,12/6/2021,Artist Oasis near Venice Beach w/ Beautiful Ga...,https://a0.muscache.com/pictures/25d657aa-2529...,9171,https://www.airbnb.com/users/show/9171,Sanni,3/5/2009,...,4.92,4.75,4.53,HSR19-002149,f,4,1,3,0,1.00


In [35]:
def popup_html(row_num, df_airbnb):
    i = row_num
    name=df_airbnb['name'].iloc[i]
    price=df_airbnb['price'].iloc[i]
    roomtype=df_airbnb['room_type'].iloc[i]
    neighbourhood_cleansed = df_airbnb['neighbourhood_cleansed'].iloc[i]


    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"

    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(name) + """

</head>
    <table style="height: 20px; width: 200px; border:1px solid black">
<tbody>
<tr style = "border:1px solid black">
<td style = "border:1px solid black">Price Per Night:</td>
<td style = "border:1px solid black">&nbsp; {}</td>""" .format(price) + """
</tr>

<tr style = "border:1px solid black">
<td style = "border:1px solid black">Room Type:</td>
<td style = "border:1px solid black">&nbsp; {}</td>""" .format(roomtype) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

In [36]:
# set price to 200 for testing
price = 100
df_airbnb_price_filtered = df_airbnb[df_airbnb.price==price]
rows = len(df_airbnb_price_filtered)
#https://georgetsilva.github.io/posts/mapping-points-with-folium/
#https://towardsdatascience.com/folium-map-how-to-create-a-table-style-pop-up-with-html-code-76903706b88a
for point in range(0,rows):
    i = point
    html = popup_html(i,df_airbnb_price_filtered)
    iframe = branca.element.IFrame(html=html,width=200,height=200)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker(airbnb_list[point], popup).add_to(mymap)

In [37]:
mymap

In [18]:
# if you want to save html locally
# mymap.save("index_LA.html") 